## Calculating likelihoods

In [1]:
import matplotlib.pyplot as plt
import corner as corner
import pandas as pd
import numpy as np
import h5py
import torch

from populations.bbh_models import get_models
import populations.bbh_models as read_models
from populations.utils.flow import NFlow
from populations.Flowsclass_dev import FlowModel
from populations import gw_obs

glasflow is using its own internal version of nflows


PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


In [2]:
#reading the pop synth file functions
def get_model_keys(path):
    alpha_val = '10'
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    f.close()

    # use only models with given alpha value
    for model in all_models:
        if 'alpha' in model:
            if 'alpha'+alpha_val in model:
                models.append('/'+model)
        else:
            models.append('/' + model)
    return(np.split(np.array(models), 5))

def get_model_keys_CE(path):
    all_models = []
    models = []
    def find_submodels(name, obj):
        if isinstance(obj, h5py.Dataset):
            all_models.append(name.rsplit('/', 1)[0])
            
    f = h5py.File(path, 'r')
    f.visititems(find_submodels)
    # get all unique models
    all_models = sorted(list(set(all_models)))
    return(np.split(np.array(models), 4))

def read_hdf5(path, all_alpha=False):
    if all_alpha:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys_CE(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])
    else:
        popsynth_outputs = {}
        models = np.asarray(get_model_keys(path))
        for i in range(models.shape[0]):
            for j in range(models.shape[1]):
                popsynth_outputs[i,j]=pd.read_hdf(path, key=models[i,j])

    return(popsynth_outputs)

In [3]:
params = ['mchirp','q', 'chieff', 'z']
param_bounds = np.array([[0,100], [0,1], [-1,1], [0,10]])
obs = np.random.rand(100,46,4)
obs[:,:,0] *=param_bounds[0,1]
obs[:,:,1] *=param_bounds[1,1]
obs[:,:,2] -= 0.5
obs[:,:,2] *=param_bounds[2,1]*2
obs[:,:,3] *=param_bounds[3,1]

In [4]:
file_path='/Users/stormcolloms/Documents/PhD/Project_work/OneChannel_Flows/models_reduced.hdf5'
gw_path = '/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/gw_events'
observations, obsdata, p_theta, events = gw_obs.generate_observations(params, gw_path, \
                                            100, 'posteriors', None)

popsynth_outputs = read_hdf5(file_path, True)
flow = FlowModel.from_samples('CE', popsynth_outputs, params)
flow.load_model('/Users/stormcolloms/Documents/PhD/Project_work/AMAZE_model_selection/flow_models/', 'CE')

In [12]:
llks = flow(obsdata,[3,1])
llks

max data scaled nan
max data scaled 0.3874934204203379
max data scaled -0.36200953349913684
[4.34497239e-14 5.46020370e-24 8.48009054e-13 6.43155260e-18
 4.79367379e-11 3.52623606e-15 6.74000864e-24 2.46898042e-14
 5.02349464e-12 1.60266482e-16 3.20050979e-13 1.38136272e-15
 3.19304045e-13 5.64534183e-12 7.30077416e-14 1.60810296e-31
 4.18412174e-16 2.55514273e-12 8.17753136e-16 3.54393218e-13
 7.78698373e-12 2.38501883e-12 2.05140995e-14 5.06302036e-23
 4.43011643e-12 3.79349503e-12 2.71841448e-24 3.08814389e-13
 1.00000000e+00 6.19593854e-16 8.25958320e-22 1.99645361e-14
 2.58142777e-20 2.10521279e-20 1.56867173e-12 1.36598442e-13
 1.69588479e-25 1.95484616e-14 2.04379295e-19 2.36740071e-13
 1.83080114e-11 2.13257750e-16 2.10890541e-17 1.90158685e-15
 2.32810367e-16 9.84665040e-13 1.07946565e-14 1.99705183e-10
 1.82146960e-15 3.03295149e-13 7.02708161e-12 7.40106949e-16
 6.45039005e-28 2.67217359e-09 4.91186605e-11 1.22852574e-14
 1.51220887e-13 2.43594489e-12 2.03079109e-19 3.549482

array([1.00000000e-02, 1.00000000e-02, 2.91675759e-05, 2.52221331e-04,
       9.00000000e-01, 3.68046621e-12, 6.33446860e-12, 1.00000000e+00,
       9.20009497e-01, 4.72628296e-03, 1.00000000e-01, 8.00000000e-02,
       1.14021354e-09, 1.18396277e-04, 7.68980017e-09, 7.35841228e-15,
       2.37049742e-07, 3.00000000e-02, 5.12760784e-05, 1.82105326e-13,
       2.16540675e-12, 1.16802978e-10, 6.83706158e-11, 2.45333758e-08,
       5.00000996e-02, 1.51636794e-11, 2.30000000e-01, 4.52697068e-01,
       2.40015751e-01, 1.80000000e-01, 2.51391284e-04, 4.00000000e-02,
       3.81420270e-04, 1.52123177e-11, 3.88195507e-12, 8.66901865e-04,
       7.30132102e-04, 7.90000000e-01, 1.34427956e-08, 1.00000000e+00,
       1.42429797e+00, 5.00000000e-02, 4.40664930e-05, 5.94959630e-06,
       6.96617301e-12, 4.15686555e-12])